In [1]:
#we are going to use spotipy to make the API calls
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

#this are spotify tokens. You need an account to get them:
cid ="50579563e8904db4a475c8c10654138c" 
secret = "b740dcff5ca9458289fa56a5f816870b"

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [2]:
# timeit library to measure the time needed to run this code
import timeit
start = timeit.default_timer()

# create empty lists where the results are going to be stored
artist_name = []
album_name = []
year = []

for i in range(0,2000, 50):
    album_results = sp.search(q='year:2020', type='album', limit=50, offset=i)
    for i, t in enumerate(album_results['albums']['items']):
        artist_name.append(t['artists'][0]['name'])
        album_name.append(t['name'])
        year.append(t['release_date'])

stop = timeit.default_timer()
print ('Time to run this code (in seconds):', stop - start)

Time to run this code (in seconds): 16.3847796


In [3]:
#then, we create the dataframe to store the information
import pandas as pd

df_albums = pd.DataFrame({'year':year,'artist_name':artist_name,'album_name':album_name})
df_albums.sort_values(by=['year'])

,year,artist_name,album_name
504,2020-01-01,Hailee Steinfeld,Wrong Direction
385,2020-01-01,Lil Tjay,20/20
1458,2020-01-01,Omer Adam,עומר
1557,2020-01-02,David Bisbal,En Tus Planes
1446,2020-01-02,Nio Garcia,La Jeepeta
280,2020-01-03,Justin Bieber,Yummy
1504,2020-01-03,Yves V,Not So Bad (feat. Emie)
1943,2020-01-03,Julie Bergan,Kiss Somebody
743,2020-01-03,Myke Towers,La Playa (Remix)
1382,2020-01-03,Ysn Fab,Made 4 More


In [4]:
#check for duplicates

grouped = df_albums.groupby(['artist_name','album_name'], as_index=True).size()
grouped[grouped > 1].count()

66

In [5]:
df_albums.drop_duplicates(subset=['artist_name','album_name'], inplace=True)

In [6]:
#stablish the connection between jupyter and Gsheets
import gspread
from oauth2client.service_account import ServiceAccountCredentials
scope = ['https://spreadsheets.google.com/feeds']
credentials = ServiceAccountCredentials.from_json_keyfile_name('./Jupyter and Google Sheets-673752ccde13.json', scope)
gc = gspread.authorize(credentials)

In [7]:
#specify the Sheet
spreadsheet_key = '1cMEmnIriNr-8a4bDykaZQuADyeNKYP72hGj93f-VMVw'
book = gc.open_by_key(spreadsheet_key)
worksheet = book.worksheet("spotify unified")
table = worksheet.get_all_values()

In [8]:
from df2gspread import df2gspread as d2g
wks_name = '2020'
d2g.upload(df_albums, spreadsheet_key, wks_name, credentials=credentials, row_names=True)

<Worksheet '2020' id:1639295153>